In [ ]:
# import classes and functions
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [ ]:
# Load ascii text and convert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [ ]:
# Prepare the data for modeling by the neural network
# we cann't model the characters directly, instead we must convert the characters to integers. 
# Step 01: creating a set of all of the distinct characters in the book
# Step 02: creating a map of each character to a unique integer

In [ ]:
# Create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
# We can see there may be some characters that we could remove to further clean up the data set that will reduce 
# the vocabulary and may improve the modeling process.
# Summarize the dataset
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

In [ ]:
# Prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns


In [ ]:
# Now we have prepared our training data 
# LSTM network requierment
# First we must transform the list of input sequences into the form
# [sample, time steps, features]
# Second we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network
# that use the sigmoid activation function by default
# Finally, we need to convert the output patterns into a one hot encoding. 
# We can configure the network to predict the probability of each of the 43 different characters in the vocabulary
# rather than trying to force it to predict precisely the next character. 

### Each y value is converted into a sparse vector with a length of 43, full of zeros except with a 1 in the column for
### the letter (integer) that the pattern represents.

In [ ]:
# Reshape X to be [Samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# Fit our model to the data. Here we use a modest number of 20 epochs and a large batch size of 128 patterns.
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)